In [ ]:
import os
import getpass
username = getpass.getuser()
os.environ['MPLCONFIGDIR'] = "~/group/c-xem2/%s/configs/" % username
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def mask(df, key, value):
    return df[df[key] == value]
pd.DataFrame.mask = mask

In [ ]:
class lowXCorr:
    def __init__(self,name):
        self.name = name
        self.model = self.getModel()
        self.A = None
        self.read_df()
        self.getAngleList()
        self.getTargList()
    
    def read_df(self):
            self.df = pd.read_csv('../sigD_sigP_10_14_20/%s.dat' % self.name, delim_whitespace =True,names=['Y','A','Z','th','Ep','x_bj','sigIN','sigQE'], \
                    dtype={'Y':np.float64,'A':np.float64,'Z':np.float64,'th':np.float64,'Ep':np.float64,'x_bj':np.float64, \
                        'sigIN':np.float64, 'sigQE':np.float64})
            self.correctUnits()
            print(self.df)
    
    def vladasFit(df):
        a1 = 0.98051
        a2 = -0.50435
        a3 = 0.18717
        a4 = 0.5813
        a5 = -0.84747
        b1 = 0.006789
        b2 = -0.11536
        b3 = 0.14625
        df['Q2'] = 4*5.766*df['Ep']*np.sin(df['th']*3.14159/180/2)*np.sin(df['th']*3.14159/180/2)
        df['A_x'] = a1 + df['x_bj']*(a2 + df['x_bj']*(a3+ df['x_bj']*(a4 + df['x_bj']*a5)))
        df['B_x'] = b1 + b2*df['x_bj'] + b3*df['x_bj']*df['x_bj']
        df['vladasFactor'] = (df['A_x'] + df['B_x']*np.log(df['Q2'])) * 2
        return

    def getAngleList(self):
        self.uAngle = np.unique(self.df['th'])
        self.uAngle = np.unique(uAngle[~np.isnan(uAngle)])
    def getTargList(self):
        self.uTarget = np.unique(self.df['A'])
        self.uTarget = np.unique(uTarget[~np.isnan(uTarget)])
            
    def correctUnits(self):
        if self.model == None:
            return
        elif self.model == 'XEM':
            self.df['sigIn'] = self.df['sigIN'] / 1000.
            self.df['sigQE'] = self.df['sigQE'] * 1000.
            self.df['sig_tot'] = self.df['sigIn'] + self.df['sigQE']
        elif self.model == 'F1F220':
            self.df['sigIn'] = self.df['sigIN'] / 1000.
            self.df['sigQE'] = self.df['sigQE'] / 1000.
            self.df['sig_tot'] = self.df['sigIn'] + self.df['sigQE']
        elif self.model == 'F1F221':
            self.df['sigIn'] = self.df['sigIN'] / 1000.
            self.df['sigQE'] = self.df['sigQE'] / 1000.
            self.df['sig_tot'] = self.df['sigIn'] + self.df['sigQE']
        else:
            print("Unknown Model!")
    
    def getAngles(self):
        return self.uAngle
    
    def getDF(self):
        return self.df()
    
    def getA(self):
        return self.A
    
    def determineTarg(self):
        if self.A == 1: return "Hydrogen"
        elif self.A == 2: return "Deuterium"
        else: return None
    
    def getModel(self):
        modelList = ['XEM','F1F220','F1F221','orig']
        for a in modelList:
            if(a in self.name):
                return a

In [ ]:
dataModelTable = pd.read_csv('../output/sigD_sigP_10_14_20/comp_d_h_full.dat',delim_whitespace =True,names=['Y','A','Z','th','Ep','x_bj','sigIN_xem20','sigQE_xem20','sigIN_xem06','sigIN_xem_o'], \
    dtype={'Y':np.float64,'A':np.float64,'Z':np.float64,'th':np.float64,'Ep':np.float64,'x_bj':np.float64,'sigIN_xem':np.float64,'sigQE_xem20':np.float64,'sigIN_xem20':np.float64,'sigQE_xem06':np.float64,'sigIN_xem_o':np.float64})
#print(dataModelTable.columns)
#print(dataModelTable)

In [ ]:
def mask(df, key, value):
    return df[df[key] == value]
pd.DataFrame.mask = mask

In [ ]:
uAngle = np.unique(dataModelTable['th'])
uAngle = np.unique(uAngle[~np.isnan(uAngle)])
uTarget = np.unique(dataModelTable['A'])
uTarget = np.unique(uTarget[~np.isnan(uTarget)])
#print(uAngle,'\n',uTarget)

In [ ]:
uIdentifier = [(target,angle) for target in uTarget for angle in uAngle]
#print(uIdentifier)

l_crossSections = []
for identifier in uIdentifier:
    tmp_df = dataModelTable.mask('A',identifier[0])
    idx = tmp_df.mask('th',identifier[1]).index
    if len(idx) > 0:
        l_crossSections.append(dataModelTable.loc[idx])
        #print(identifier[0], identifier[1], '\t\t', len(l_crossSections))

In [ ]:
def mask2(df, key, value):
    return df[df[key] >= value ]
pd.DataFrame.mask = mask2

In [ ]:
l_crossSectionsCut = []
for df in l_crossSections:
    idx = df.mask('x_bj',0).index
    l_crossSectionsCut.append(df.loc[idx])
for df in l_crossSectionsCut:
    df['sigInXEM20'] = df['sigIN_xem20'] / 1000.

    df['sigInXEM06'] = df['sigIN_xem06'] / 1000.
    
    df['sigInXEM_o'] = df['sigIN_xem_o'] / 1000.


In [ ]:
newXEMmodel = pd.read_csv('../output/sigD_sigP_10_14_20/comp_d_h_new.dat',delim_whitespace =True,names=['Y','A','Z','th','Ep','x_bj','sigIN_xem21','sigQE_xem21'], \
    dtype={'Y':np.float64,'A':np.float64,'Z':np.float64,'th':np.float64,'Ep':np.float64,'x_bj':np.float64,'sigQE_xem21':np.float64,'sigIN_xem21':np.float64,})
#print(newXEMmodel.columns)
#print(newXEMmodel)

In [ ]:
uAngle2 = np.unique(newXEMmodel['th'])
uAngle2 = np.unique(uAngle2[~np.isnan(uAngle2)])
uTarget2 = np.unique(newXEMmodel['A'])
uTarget2 = np.unique(uTarget2[~np.isnan(uTarget2)])
#print(uAngle2,'\n',uTarget2)

In [ ]:
def mask(df, key, value):
    return df[df[key] == value]
pd.DataFrame.mask = mask

In [ ]:
uIdentifier2 = [(target2,angle2) for target2 in uTarget2 for angle2 in uAngle2]
#print(uIdentifier)

l_crossSections2 = []
for identifier2 in uIdentifier2:
    tmp_df2 = newXEMmodel.mask('A',identifier2[0])
    idx2 = tmp_df2.mask('th',identifier2[1]).index
    if len(idx2) > 0:
        l_crossSections2.append(newXEMmodel.loc[idx2])
        #print(identifier2[0], identifier2[1], '\t\t', len(l_crossSections2))

In [ ]:
def mask2(df, key, value):
    return df[df[key] >= value ]
pd.DataFrame.mask = mask2

In [ ]:
l_crossSectionsCut2 = []
for df2 in l_crossSections2:
    idx2 = df2.mask('x_bj',0).index
    l_crossSectionsCut2.append(df2.loc[idx2])
for df2 in l_crossSectionsCut2:
    df2['sigInXEM21'] = df2['sigIN_xem21'] / 1000.
    #print(df2)

In [ ]:
def determineTarg(A):
    if A == 1: return "Hydrogen"
    elif A == 2: return "Deuterium"
    elif A == 3: return "Helium3"
    elif A == 4: return "Helium4"
    elif A == 9: return "Beryllium"
    elif A == 12: return "Carbon"
    elif A == 64: return "Copper"
    elif A == 197: return "Gold"
    else: return "CRAP"

In [ ]:
def my_plotter(df):
    xmin=0.3
    xmax=0
    ymin=1e-9
    ymax=1e1
    doLog = True
    fig, ax1 = plt.subplots(2,1,figsize=(15, 2*7))

    if len(df['A']) > 0.:
        targ = determineTarg(df['A'].iloc[0])
        ax1[0].set_title('%s, at %2.0f degrees' % (targ, df['th'].iloc[0]))
        ax1[1].set_title('%s, at %2.0f degrees' % (targ, df['th'].iloc[0]))

        xmax = 2. if (df['x_bj'].iloc[-1] + 0.2 > 2.) else df['x_bj'].iloc[-1] + 0.1
        #xmax=3
        if(df['A'].iloc[0] < 3):
            xmax=2
    #Models and Data Comparisons
    #df.plot.scatter('x_bj','CS',c='magenta',s=36,marker='o',linestyle='None',yerr='CS_stat',xlim=(xmin,xmax),ylim=(ymin,ymax),ax=ax1[0],logy=doLog, label='Cross-Section')
    #df.plot.scatter('x_bj','sigQEXEM',c='red',s=16,marker='v',linestyle='None',xlim=(xmin,xmax),ylim=(ymin,ymax),ax=ax1[0],logy=doLog, label='QE XEM Model')
    df.plot.scatter('x_bj','sigInXEM20',c='blue',s=16,marker='^',linestyle='None',xlim=(xmin,xmax),ylim=(ymin,ymax),ax=ax1[0],logy=doLog, label='DIS XEM Model with F1F2in20')
    df.plot.scatter('x_bj','sigInXEM06',c='green',s=32,marker='^',linestyle='None',xlim=(xmin,xmax),ylim=(ymin,ymax),ax=ax1[0],logy=doLog, label='DIS XEM Model with F1F2in06')
    #df.plot.scatter('x_bj','sigTotXEM',c='green',s=16,marker='o',linestyle='None',xlim=(xmin,xmax),ylim=(ymin,ymax),ax=ax1[0],logy=doLog, label='XEM Model')
    
    #Ratio plot
    df.plot.scatter('x_bj','data_ratioXEM20_06',c='red',marker='o',linestyle='None',xlim=(xmin,xmax),ylim=(0.75,1.25),logy=False,ax=ax1[1])
    ax1[0].set_ylabel("Cross Section")

    plt.savefig('%s_at_%2.0fdeg.pdf' % (targ, df['th'].iloc[0]),bbox_inches='tight')
    ax1[1].set_ylabel("Ratio")

    
    return

In [ ]:
#nPlots = len(l_crossSectionsCut)*2
#print(nPlots)

#my_plotter(l_crossSectionsCut[0])

#for df in l_crossSectionsCut:
    #my_plotter(df)
    #print(df[['sigInXEM06','sigInXEM_o','sigInXEM20','x_bj','A','Z','th']])
#for df2 in l_crossSectionsCut2:
    #my_plotter(df)
    #print(df2[['sigInXEM21','x_bj','A','Z','th']])

In [ ]:
def vladasFit(df):
    a1 = 0.98051
    a2 = -0.50435
    a3 = 0.18717
    a4 = 0.5813
    a5 = -0.84747
    b1 = 0.006789
    b2 = -0.11536
    b3 = 0.14625
    df['Q2'] = 4*5.766*df['Ep']*np.sin(df['th']*3.14159/180/2)*np.sin(df['th']*3.14159/180/2)
    df['A_x'] = a1 + df['x_bj']*(a2 + df['x_bj']*(a3+ df['x_bj']*(a4 + df['x_bj']*a5)))
    df['B_x'] = b1 + b2*df['x_bj'] + b3*df['x_bj']*df['x_bj']
    df['vladasFactor'] = (df['A_x'] + df['B_x']*np.log(df['Q2'])) * 2
    return

In [ ]:
#for df in l_crossSectionsCut:
    #print('A:',df['A'].iloc[0], 'theta:',df['th'].iloc[0])

fourtyDegCompF1F2in20_0 = pd.DataFrame() 
fourtyDegCompF1F2in20 = pd.DataFrame() 
fourtyDegCompF1F2in06 = pd.DataFrame() 
fourtyDegCompF1F2in21 = pd.DataFrame() 

fiftyDegCompF1F2in20_0 = pd.DataFrame() 
fiftyDegCompF1F2in20 = pd.DataFrame() 
fiftyDegCompF1F2in06 = pd.DataFrame()
fiftyDegCompF1F2in21 = pd.DataFrame()

fourtyDegCompF1F2in20_0['th'] = l_crossSectionsCut[0]['th']
fourtyDegCompF1F2in20_0['Ep'] = l_crossSectionsCut[0]['Ep']

fiftyDegCompF1F2in20_0['th'] = l_crossSectionsCut[3]['th']
fiftyDegCompF1F2in20_0['Ep'] = l_crossSectionsCut[3]['Ep']


fourtyDegCompF1F2in20_0['x_bj'] = l_crossSectionsCut[0]['x_bj']
fourtyDegCompF1F2in20['x_bj'] = l_crossSectionsCut[0]['x_bj']
fourtyDegCompF1F2in21['x_bj'] = l_crossSectionsCut2[0]['x_bj']
fourtyDegCompF1F2in06['x_bj'] = l_crossSectionsCut[0]['x_bj']

fiftyDegCompF1F2in20_0['x_bj'] = l_crossSectionsCut[3]['x_bj']
fiftyDegCompF1F2in20['x_bj'] = l_crossSectionsCut[3]['x_bj']
fiftyDegCompF1F2in21['x_bj'] = l_crossSectionsCut2[3]['x_bj']
fiftyDegCompF1F2in06['x_bj'] = l_crossSectionsCut[3]['x_bj']

fourtyDegCompF1F2in20_0['sigD_sigP_Ratio20_0'] = l_crossSectionsCut[2]['sigInXEM_o'].to_numpy() / l_crossSectionsCut[0]['sigInXEM_o'].to_numpy();
fourtyDegCompF1F2in20_0['sigD_sigP_Ratio20_0'] = l_crossSectionsCut[2]['sigInXEM_o'].to_numpy() / l_crossSectionsCut[0]['sigInXEM_o'].to_numpy();
fourtyDegCompF1F2in20['sigD_sigP_Ratio20'] = l_crossSectionsCut[2]['sigInXEM20'].to_numpy()*1000. / l_crossSectionsCut[0]['sigInXEM20'].to_numpy();
fourtyDegCompF1F2in06['sigD_sigP_Ratio06'] = l_crossSectionsCut[2]['sigInXEM06'].to_numpy()*1000. / l_crossSectionsCut[0]['sigInXEM06'].to_numpy();
fourtyDegCompF1F2in20['sigD_sigP_Ratio20'] = l_crossSectionsCut[2]['sigInXEM20'].to_numpy()*1000. / l_crossSectionsCut[0]['sigInXEM20'].to_numpy();
fourtyDegCompF1F2in06['sigD_sigP_Ratio06'] = l_crossSectionsCut[2]['sigInXEM06'].to_numpy()*1000. / l_crossSectionsCut[0]['sigInXEM06'].to_numpy();

fourtyDegCompF1F2in21['sigD_sigP_Ratio21'] = l_crossSectionsCut2[2]['sigInXEM21'].to_numpy()*10. / l_crossSectionsCut2[0]['sigInXEM21'].to_numpy();
fourtyDegCompF1F2in21['sigD_sigP_Ratio21'] = l_crossSectionsCut2[2]['sigInXEM21'].to_numpy()*10. / l_crossSectionsCut2[0]['sigInXEM21'].to_numpy();
vladasFit(fourtyDegCompF1F2in20_0)

fiftyDegCompF1F2in20_0['sigD_sigP_Ratio20_0'] = l_crossSectionsCut[3]['sigInXEM_o'].to_numpy() / l_crossSectionsCut[1]['sigInXEM_o'].to_numpy();
fiftyDegCompF1F2in20_0['sigD_sigP_Ratio20_0'] = l_crossSectionsCut[3]['sigInXEM_o'].to_numpy() / l_crossSectionsCut[1]['sigInXEM_o'].to_numpy();
fiftyDegCompF1F2in20['sigD_sigP_Ratio20'] = l_crossSectionsCut[3]['sigInXEM20'].to_numpy()*1000. / l_crossSectionsCut[1]['sigInXEM20'].to_numpy();
fiftyDegCompF1F2in20['sigD_sigP_Ratio20'] = l_crossSectionsCut[3]['sigInXEM20'].to_numpy()*1000. / l_crossSectionsCut[1]['sigInXEM20'].to_numpy();
fiftyDegCompF1F2in06['sigD_sigP_Ratio06'] = l_crossSectionsCut[3]['sigInXEM06'].to_numpy()*1000. / l_crossSectionsCut[1]['sigInXEM06'].to_numpy();
fiftyDegCompF1F2in06['sigD_sigP_Ratio06'] = l_crossSectionsCut[3]['sigInXEM06'].to_numpy()*1000. / l_crossSectionsCut[1]['sigInXEM06'].to_numpy();

fiftyDegCompF1F2in21['sigD_sigP_Ratio21'] = l_crossSectionsCut2[3]['sigInXEM21'].to_numpy()*10. / l_crossSectionsCut2[1]['sigInXEM21'].to_numpy();
fiftyDegCompF1F2in21['sigD_sigP_Ratio21'] = l_crossSectionsCut2[3]['sigInXEM21'].to_numpy()*10. / l_crossSectionsCut2[1]['sigInXEM21'].to_numpy();
vladasFit(fiftyDegCompF1F2in20_0)

In [ ]:
xmin=0.15
xmax=1.
doLog = True
fig, ax1 = plt.subplots(2,1,figsize=(15, 2*7))

fourtyDegCompF1F2in20_0.plot('x_bj','vladasFactor',c='black',xlim=(xmin,xmax),
        ylim=(0.75,2),logy=False,ax=ax1[0], label='sigD / sigP Vladas')
fourtyDegCompF1F2in20.plot.scatter('x_bj','sigD_sigP_Ratio20',c='red',marker='o',
        linestyle='None',xlim=(xmin,xmax),ylim=(0.75,2),logy=False,ax=ax1[0], label='sigD / sigP F1F220')
fourtyDegCompF1F2in21.plot.scatter('x_bj','sigD_sigP_Ratio21',c='green',marker='o',
        linestyle='None',xlim=(xmin,xmax),ylim=(0.75,2),logy=False,ax=ax1[0], label='sigD / sigP F1F221')
fourtyDegCompF1F2in06.plot.scatter('x_bj','sigD_sigP_Ratio06',c='magenta',marker='o',
        linestyle='None',xlim=(xmin,xmax),ylim=(0.75,2),logy=False,ax=ax1[0], label='sigD / sigP F1F206 + hack')
fourtyDegCompF1F2in20_0.plot.scatter('x_bj','sigD_sigP_Ratio20_0',c='blue',marker='o',
        linestyle='None',xlim=(xmin,xmax),ylim=(0.75,2),logy=False,ax=ax1[0], label='sigD / sigP F1F2_standalone')
ax1[0].set_title("Fourty Degree Comparison")
ax1[0].set_ylabel("SigD / SigP")
ax1[0].set_xlabel("X")

fiftyDegCompF1F2in20_0.plot('x_bj','vladasFactor',color='black',xlim=(xmin,xmax),
    ylim=(0.75,2),logy=False,ax=ax1[1], label='sigD / sigP Vladas')
fiftyDegCompF1F2in20.plot.scatter('x_bj','sigD_sigP_Ratio20',color='red',marker='o',
    linestyle='None',xlim=(xmin,xmax),ylim=(0.75,2),logy=False,ax=ax1[1], label='sigD / sigP F1F220')
fiftyDegCompF1F2in21.plot.scatter('x_bj','sigD_sigP_Ratio21',color='green',marker='o',
    linestyle='None',xlim=(xmin,xmax),ylim=(0.75,2),logy=False,ax=ax1[1], label='sigD / sigP F1F221')
fiftyDegCompF1F2in20_0.plot.scatter('x_bj','sigD_sigP_Ratio20_0',color='blue',marker='o',
    linestyle='None',xlim=(xmin,xmax),ylim=(0.75,2),logy=False,ax=ax1[1], label='sigD / sigP F1F2_standalone')
fiftyDegCompF1F2in06.plot.scatter('x_bj','sigD_sigP_Ratio06',color='magenta',marker='o',
    linestyle='None',xlim=(xmin,xmax),ylim=(0.75,2),logy=False,ax=ax1[1], label='sigD / sigP F1F206 + hack')
ax1[1].set_title("Fifty Degree Comparison")
ax1[1].set_ylabel("SigD / SigP")
ax1[1].set_xlabel("X")

In [ ]:
xmin=0.15
xmax=1.
doLog = True
fig, ax1 = plt.subplots(2,1,figsize=(15, 2*7))

fourtyDegCompF1F2in20_0.plot('x_bj','vladasFactor',c='black',xlim=(xmin,xmax),
        ylim=(0.75,2),logy=False,ax=ax1[0], label='sigD / sigP Vladas')
fourtyDegCompF1F2in20.plot.scatter('x_bj','sigD_sigP_Ratio20',c='red',marker='o',
        linestyle='None',xlim=(xmin,xmax),ylim=(0.75,2),logy=False,ax=ax1[0], label='sigD / sigP F1F220')
fourtyDegCompF1F2in21.plot.scatter('x_bj','sigD_sigP_Ratio21',c='green',marker='o',
        linestyle='None',xlim=(xmin,xmax),ylim=(0.75,2),logy=False,ax=ax1[0], label='sigD / sigP F1F221')
ax1[0].set_title("Fourty Degree Comparison")
ax1[0].set_ylabel("SigD / SigP")
ax1[0].set_xlabel("X")

fiftyDegCompF1F2in20_0.plot('x_bj','vladasFactor',color='black',xlim=(xmin,xmax),
    ylim=(0.75,2),logy=False,ax=ax1[1], label='sigD / sigP Vladas')
fiftyDegCompF1F2in20.plot.scatter('x_bj','sigD_sigP_Ratio20',color='red',marker='o',
    linestyle='None',xlim=(xmin,xmax),ylim=(0.75,2),logy=False,ax=ax1[1], label='sigD / sigP F1F220')
fiftyDegCompF1F2in21.plot.scatter('x_bj','sigD_sigP_Ratio21',color='green',marker='o',
    linestyle='None',xlim=(xmin,xmax),ylim=(0.75,2),logy=False,ax=ax1[1], label='sigD / sigP F1F221')
ax1[1].set_title("Fifty Degree Comparison")
ax1[1].set_ylabel("SigD / SigP")
ax1[1].set_xlabel("X")